In [4]:
library(tidyverse)
library(tidymodels)
library(repr)
library(rvest)
library(stringr)
options(repr.matrix.max.rows = 6)
set.seed(10)


Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding




Packages containing functions required for the following classification model

In [2]:
cleveland_data<-read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data")
colnames(cleveland_data)<-c("age","sex","chest_pain","rest_bps","cholestrol_level","high_fbs?",
                            "rest_electrographics","max_heartrate","exercise_induced_angina","oldpeak","slope",
                            "colored_major_vessels","thal","risk_level")
                          
cleveland_data<-cleveland_data|>
                    mutate(exercise_induced_angina=case_when(exercise_induced_angina==0~"No",exercise_induced_angina==1~"Yes"))|>
                    mutate(sex=case_when(sex==0~"female",sex==1~"male"))
cleveland_data

New names:
• `1.0` -> `1.0...2`
• `1.0` -> `1.0...3`
• `1.0` -> `1.0...6`
• `0.0` -> `0.0...9`
• `0.0` -> `0.0...12`
Rows: 302 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): 0.0...12, 6.0
dbl (12): 63.0, 1.0...2, 1.0...3, 145.0, 233.0, 1.0...6, 2.0, 150.0, 0.0...9...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,chest_pain,rest_bps,cholestrol_level,high_fbs?,rest_electrographics,max_heartrate,exercise_induced_angina,oldpeak,slope,colored_major_vessels,thal,risk_level
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
67,male,4,160,286,0,2,108,Yes,1.5,2,3.0,3.0,2
67,male,4,120,229,0,2,129,Yes,2.6,2,2.0,7.0,1
37,male,3,130,250,0,0,187,No,3.5,3,0.0,3.0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,male,4,130,131,0,0,115,Yes,1.2,2,1.0,7.0,3
57,female,2,130,236,0,2,174,No,0.0,2,1.0,3.0,1
38,male,3,138,175,0,0,173,No,0.0,1,?,3.0,0


Processed Cleveland Heart Dataset: 
-obtained from UCI Machine Learning Repository-Center for Machine Learning and Intelligent Systems
-data cleaned and wrangled, changed variables to human readable names.
Note: generally 0=normal and 1-4=abnormal, presence of, or potential risk

In [4]:
set.seed(1111)

cleveland_split <- initial_split(cleveland_data, prop = 0.75, strata = risk_level)
cleveland_train <- training(cleveland_split)
cleveland_test <- testing(cleveland_split)

cleveland_train
cleveland_test

age,sex,chest_pain,rest_bps,cholestrol_level,high_fbs?,rest_electrographics,max_heartrate,exercise_induced_angina,oldpeak,slope,colored_major_vessels,thal,risk_level
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
37,male,3,130,250,0,0,187,No,3.5,3,0.0,3.0,0
41,female,2,130,204,0,2,172,No,1.4,1,0.0,3.0,0
57,female,4,120,354,0,0,163,Yes,0.6,1,0.0,3.0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
61,male,4,138,166,0,2,125,Yes,3.6,2,1.0,3.0,4
55,female,4,128,205,0,1,130,Yes,2.0,2,1.0,7.0,3
59,male,4,164,176,1,2,90,No,1.0,2,2.0,6.0,3


age,sex,chest_pain,rest_bps,cholestrol_level,high_fbs?,rest_electrographics,max_heartrate,exercise_induced_angina,oldpeak,slope,colored_major_vessels,thal,risk_level
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
56,male,2,120,236,0,0,178,No,0.8,1,0.0,3.0,0
57,male,4,140,192,0,0,148,No,0.4,2,0.0,6.0,0
56,male,3,130,256,1,2,142,Yes,0.6,2,1.0,6.0,2
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
35,male,2,122,192,0,0,174,No,0.0,1,0.0,3.0,0
58,male,4,114,318,0,1,140,No,4.4,3,3.0,6.0,4
57,male,4,130,131,0,0,115,Yes,1.2,2,1.0,7.0,3


Split original dataset to Training(75%) and Testing(25%). Training data will be used to design predictive model (classifier) in the following code.